## 导入库

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import tqdm
import time
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm
from sklearn.utils import shuffle
from skimage.io import imread
from skimage.transform import pyramid_gaussian, integral_image
from PIL import Image

%matplotlib inline

## 参数设置

In [2]:
class Config():
    def __init__(self, project_id=None):
        if project_id:
            self.project_id = project_id
        else:
            self.project_id = "PROJECT_ORB_RGB"
        
        ## 路径参数
        self.set_path()
        self.check_path()
        
        ## ORB特征参数
        self.ORBParam = {
            "nfeatures" : 200,   # 提取最多特征点的数量
            "scales" : 1.2,      # 金字塔图像之间的尺寸参数
            "nlevels" : 8,       # 高斯金字塔的层数
            "edgethresh" : 2,   # 主要根据patchsize来决定检测多少邻域
            "wetk" : 2,          # 产生BIREF描述子的点对个数，默认一般为2个
            #"scoretype" : cv2.ORB_HARRIS_SCORE,
            "patchsize" : 10     # 用于计算BIREF描述子特征点邻域大小
        }
        
        ## K-means聚类参数
        self.minibatch = True      # 是否使用minibatchkmeans
        self.Kmeans = {
            "n" : 128,               # 聚类簇数
            "init" : 'k-means++',   # 初始值选择算法
            "n_init" : 10,          # 重新选择初始值的次数
            "max_iter" : 300,       # 单次初始值计算的最大迭代次数 
            "tol" : 0.0001,         # 容忍度，kmeans运行准则收敛条件
            "n_jobs" : 1,           # 并行数
            "algorithm":'auto',     # k-means实现算法
            "batch_size":10000        # MiniBatchKmeans的batchsize
        }
        
        
        ## 颜色直方图特征参数
        self.RGBParam = {
            "channels" : 3,
            "dims" : 3
        }
        
    def set_path(self):
        ## 设置路径参数
        self.DirPath = {
            "POS_DIR_PATH" : "./source/images/pos",      # 正样本路径
            "NEG_DIR_PATH" : "./source/images/neg_new",  # 负样本路径
            "TEST_DIR_PATH" : "./source/test_images",    # 测试样本路径
            "FEAT_EXTRACTOR_PATH" : os.path.join("./source/extractor", self.project_id), # 保存图像的特征extractor
            "MODEL_DIR_PATH" : os.path.join("./source/models", self.project_id), # 保存模型的路径
            "FEAT_DIR_PATH" : os.path.join("./source/features", self.project_id), # 保存将样本转化为features的文件路径
            "PRED_SAVE_PATH" : os.path.join("./source/predictions", self.project_id) # 保存预测结果路径
        }
    def check_path(self):
        ## 检查路径是否存在
        for path in self.DirPath.values():
            if not os.path.exists(path):
                os.makedirs(path)
                print("成功新建文件夹: ", path)
##############
config = Config()

## 使用ORB特征点思路

+ Step1：使用ORB特征点检测函数检测特征点并生成特征点的描述子。
+ Step2：使用K-means对特征点向量进行聚类。
+ Step3：使用聚类和SVM训练特征点到图像的分类关系。

## 使用颜色直方图思路

+ Step1：统计颜色直方图，按照某一分度将颜色划分成等长的区域。
+ Step2：统计直方图生成颜色直方图向量，并归一化。
+ Step3：对直方图向量使用SVM进行训练。

## 实现ORB特征生成和聚类

In [4]:
class ORBExtractor():
    def __init__(self, config):
        self.config = config
        ## 初始化ORB特征点生成器
        print("初始化ORB")
        self.orb = cv2.ORB_create(nfeatures=config.ORBParam["nfeatures"],
                             scaleFactor=config.ORBParam["scales"],
                             nlevels=config.ORBParam["nlevels"],
                             edgeThreshold=config.ORBParam["edgethresh"],
                             WTA_K=config.ORBParam["wetk"],
                             scoreType=cv2.ORB_HARRIS_SCORE,
                             patchSize=config.ORBParam["patchsize"])
        
    def ExtractORB(self, image):
        '''
        func : 输入一张图片，返回该图片的特征点的向量.
        param :
            image : 输入图片
        return : 二维数组，shape(kp_num, feature_dim)
        '''
        kp, des = self.orb.detectAndCompute(image, None)
        return kp, des
    
    def DrawKeyPoints(self, keypoints, image):
        '''
        func : 将特征点画在图像上并显示
        param : 
            image : 输入图像
            keypoints : 通过ORB提取的特征点
        return :
            None
        '''
        img = cv2.drawKeypoints(image, keypoints, None, (0,0,255), 0)
        plt.imshow(img)
####################
extractor = ORBExtractor(config)
## 保存ORBExtractor
#joblib.dump(extractor, os.path.join(config.DirPath["FEAT_EXTRACTOR_PATH"], config.project_id+"_feat_extractor.pkl"))

初始化ORB


In [4]:
## 从图片中Extract出特征点向量
PosImageDir = config.DirPath["POS_DIR_PATH"]
NegImageDir = config.DirPath["NEG_DIR_PATH"]
KeyPointList = []  # 用于保有存所的关键点

## 循环遍历图片路径并extract出关键点
print("正样本图像关键点提取......")
for path in tqdm.tqdm(os.listdir(PosImageDir)):
    image = cv2.imread(os.path.join(PosImageDir, path), 0)
    KeyPointList.append(extractor.ExtractORB(image)[1])
print("负样本图像关键点提取......")
for path in tqdm.tqdm(os.listdir(NegImageDir)):
    image = cv2.imread(os.path.join(NegImageDir, path), 0)
    KeyPointList.append(extractor.ExtractORB(image)[1])

## 合并这些特征点向量
print("合并特征点向量")
def notNone(x):
    if type(x) == type(None):
        return False
    else:
        return True
KeyPointList = list(filter(notNone, KeyPointList)) # 过滤掉None
PointFeatures = np.concatenate(tuple(KeyPointList), axis=0)
np.random.shuffle(PointFeatures) # 对点的特征向量进行聚类
PointFeaturesNorm = 1.0*(PointFeatures - PointFeatures.mean()) / PointFeatures.std()  # 对数据标准化

## 对这些特征向量进行聚类
start_time = time.time()
if not config.minibatch:
    print("使用K-means进行聚类......")
    km = KMeans(n_clusters=config.Kmeans["n"],
                       init=config.Kmeans["init"],
                       n_init=config.Kmeans["n_init"],
                       max_iter=config.Kmeans["max_iter"],
                       tol=config.Kmeans["tol"],
                       n_jobs=config.Kmeans["n_jobs"],
                       algorithm=config.Kmeans["algorithm"])
    km.fit(PointFeaturesNorm) # 聚类
else:
    print("使用MinibatchKmeans聚类......")
    km = MiniBatchKMeans(n_clusters=config.Kmeans["n"],
                       init=config.Kmeans["init"],
                       n_init=config.Kmeans["n_init"],
                       max_iter=config.Kmeans["max_iter"],
                       batch_size=config.Kmeans["batch_size"],
                       tol=config.Kmeans["tol"])
    km.fit(PointFeaturesNorm)
end_time = time.time()
print("==> 耗费时间 : {}s".format(end_time-start_time))

## 保存k-means的模型
#print("保存K-means模型......")
#joblib.dump(km, os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans.pkl"))
#param = {"mean" : PointFeatures.mean(), "std" : PointFeatures.std()}
#joblib.dump(param, os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans_param.pkl"))

  0%|          | 0/9731 [00:00<?, ?it/s]

正样本图像关键点提取......


  0%|          | 5/6432 [00:00<02:35, 41.27it/s]

负样本图像关键点提取......


100%|██████████| 6432/6432 [00:26<00:00, 245.94it/s]


合并特征点向量
使用MinibatchKmeans聚类......
==> 耗费时间 : 12.249191522598267s


## 使用聚类器预测特征点

In [5]:
## 读取K-means聚类器
print("Load k-means......")
## 载入参数
km = joblib.load(os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans.pkl"))
param = joblib.load(os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans_param.pkl"))
mean = param["mean"]
std = param["std"]
## 读取训练数据
PosImageDirs = [config.DirPath["POS_DIR_PATH"], "./source/images/no_hands/pos/"]
NegImageDirs = [config.DirPath["NEG_DIR_PATH"], "./source/images/neg/"]
PosSamples = []  # 用于保存正样本
NegSamples = []  # 用于保存负样本

def GetPosNegSamples(PosImageDirs, NegImageDirs):
    '''
    func : 遍历样本图片生成ORB特征
    param : 
        PosImageDirs : 正样本图片文件夹路径列表
        NegImageDirs : 负样本图片文件夹路径列表
    return :
        None
    '''
    print("正样本图像关键点提取......")
    for dir_name in PosImageDirs:
        for path in tqdm.tqdm(os.listdir(dir_name)):
            image = cv2.imread(os.path.join(dir_name, path), 0)
            feat = extractor.ExtractORB(image)[1] # 得到图片的ORB特征点
            if type(feat) == type(None): # 如果没有找到特征点
                PosSamples.append(np.array([]))
                continue
            feat = 1.0*(feat-mean)/std
            feat = km.predict(feat) # 使用km对特征点进行归类
            PosSamples.append(feat)
    print("负样本图像关键点提取......")
    for dir_name in NegImageDirs:
        for path in tqdm.tqdm(os.listdir(dir_name)):
            image = cv2.imread(os.path.join(dir_name, path), 0)
            feat = extractor.ExtractORB(image)[1] # 得到图片的ORB特征点
            if type(feat) == type(None): # 如果没有找到特征点
                NegSamples.append(np.array([]))
                continue
            feat = 1.0*(feat-mean)/std
            feat = km.predict(feat) # 使用km对特征点进行归类
            NegSamples.append(feat)

GetPosNegSamples(PosImageDirs, NegImageDirs)
## 保存生成的特征
joblib.dump(PosSamples, os.path.join(config.DirPath["FEAT_DIR_PATH"], config.project_id+"_pos_features.pkl"))
joblib.dump(NegSamples, os.path.join(config.DirPath["FEAT_DIR_PATH"], config.project_id+"_neg_features.pkl"))

Load k-means......


  0%|          | 17/9731 [00:00<00:59, 162.65it/s]

正样本图像关键点提取......


  1%|▏         | 87/6432 [00:00<00:07, 865.72it/s]

负样本图像关键点提取......


100%|██████████| 2000/2000 [00:06<00:00, 320.20it/s]


['./source/features/PROJECT_ORB_RGB/PROJECT_ORB_RGB_neg_features.pkl']

## 将feature转化为统计向量和SVM训练

In [ ]:
## 用于将一个特征类别向量转化为统计向量
n_range = config.Kmeans["n"]
def GenOneHot(arr):
    oh = np.zeros((n_range,))
    for i in arr:
        oh[i] += 1
    return oh
## 载入features
print("载入features......")
PosSamples = joblib.load(os.path.join(config.DirPath["FEAT_DIR_PATH"], config.project_id+"_pos_features.pkl"))
NegSamples = joblib.load(os.path.join(config.DirPath["FEAT_DIR_PATH"], config.project_id+"_neg_features.pkl"))
## 将向量转化为统计向量
PosSamples = list(map(GenOneHot, PosSamples))
NegSamples = list(map(GenOneHot, NegSamples))
X = np.concatenate([PosSamples, NegSamples], axis=0) # 连接正负样本
Y = np.concatenate([np.array([1]*len(PosSamples)), np.array([0]*len(NegSamples))])
print("总共的样本个数：{}，正样本个数：{}，负样本个数：{}".format(len(X), len(PosSamples), len(NegSamples)))

## shuffle and train
print("训练SVM......")
X, Y = shuffle(X, Y, random_state=0)
clf = svm.SVC(C=1.0,
              kernel="linear",
              probability=True)
clf.fit(X, Y)

## 将SVM模型保存
#print("保存SVM模型......")
#joblib.dump(clf, os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_svm.pkl"))

In [9]:
## 将SVM模型保存
print("保存SVM模型......")
joblib.dump(clf, os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_svm.pkl"))

保存SVM模型......


['./source/models/PROJECT_ORB_RGB/PROJECT_ORB_RGB_svm.pkl']

## 构建分类函数

In [5]:
## 用于将一个特征类别向量转化为统计向量
n_range = config.Kmeans["n"]
def GenOneHot(arr):
    oh = np.zeros((n_range,))
    for i in arr:
        oh[i] += 1
    return oh

In [6]:
## 得到extractor，image->features
# extractor
## 得到k-means分类器，features->clusters
print("Load K-means......")
km = joblib.load(os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans.pkl"))
param = joblib.load(os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_kmeans_param.pkl"))
mean = param["mean"]
std = param["std"]
## 得到svm二分类器，clusters->statistics vector->result
#print("Load SVM......")
clf = joblib.load(os.path.join(config.DirPath["MODEL_DIR_PATH"], config.project_id+"_svm.pkl"))

n_range = config.Kmeans["n"]
def Classifier(image, score=False):
    '''
    func : 用于对输入的图片进行分类是否含有卡片
    param :
        image : 输入图像
    return :
        False : 无卡片
        True : 有卡片
    '''
    feat = extractor.ExtractORB(image)[1] # 从图像提取点特征
    if type(feat) == type(None):  # 如果提取不出特征
        feat = np.array([0]*config.Kmeans["n"])
    else:
        feat = 1.0*(feat-mean)/std # 归一化
        feat = km.predict(feat) # 把特征点进行归类到簇
        feat = GenOneHot(feat) # 将特征点向量转化成统计向量
    if not score:
#         pred = clf.predict([feat]) # 使用SVM进行预测
        score = clf.decision_function([feat])[0]
        if score > 1.5:
            return True
        else:
            return False
    else:
        return clf.decision_function([feat])[0]

Load K-means......


In [9]:
## Testing
img = cv2.imread("./source/images/neg/2.jpg", 0)
Classifier(img, True)

NameError: name 'extractor' is not defined

## 进行滑动窗口分类

In [7]:
def overlapping_area(detection_1, detection_2):
    """
        Function to calculate overlapping area'si
        `detection_1` and `detection_2` are 2 detections whose area
        of overlap needs to be found out.
        Each detection is list in the format ->
        [x-top-left, y-top-left, confidence-of-detections, width-of-detection, height-of-detection]
        The function returns a value between 0 and 1,
        which represents the area of overlap.
        0 is no overlap and 1 is complete overlap.
        Area calculated from ->
        http://math.stackexchange.com/questions/99565/simplest-way-to-calculate-the-intersect-area-of-two-rectangles
    """
    # Calculate the x-y co-ordinates of the rectangles
    x1_tl = detection_1[0]
    x2_tl = detection_2[0]
    x1_br = detection_1[0] + detection_1[3]
    x2_br = detection_2[0] + detection_2[3]
    y1_tl = detection_1[1]
    y2_tl = detection_2[1]
    y1_br = detection_1[1] + detection_1[4]
    y2_br = detection_2[1] + detection_2[4]
    # Calculate the overlapping Area
    x_overlap = max(0, min(x1_br, x2_br) - max(x1_tl, x2_tl))
    y_overlap = max(0, min(y1_br, y2_br) - max(y1_tl, y2_tl))
    overlap_area = x_overlap * y_overlap
    area_1 = detection_1[3] * detection_2[4]
    area_2 = detection_2[3] * detection_2[4]
    total_area = area_1 + area_2 - overlap_area
    return overlap_area / float(total_area)

def nms(detections, threshold=.5):
    """
        This function performs Non-Maxima Suppression.
        `detections` consists of a list of detections.
        Each detection is in the format ->
        [x-top-left, y-top-left, confidence-of-detections, width-of-detection, height-of-detection]
        If the area of overlap is greater than the `threshold`,
        the area with the lower confidence score is removed.
        The output is a list of detections.
    """
    if len(detections) == 0:
        return []
    # Sort the detections based on confidence score
    detections = sorted(detections, key=lambda detections: detections[2],
                        reverse=True)
    new_detections = [] # Unique detections will be appended to this list
    new_detections.append(detections[0]) # Append the first detection
    del detections[0] # Remove the detection from the original list
    """
        For each detection, calculate the overlapping area
        and if area of overlap is less than the threshold set
        for the detections in `new_detections`, append the 
        detection to `new_detections`.
        In either case, remove the detection from `detections` list.
    """
    for index, detection in enumerate(detections):
        for new_detection in new_detections:
            if overlapping_area(detection, new_detection) > threshold:
                del detections[index]
                break
        else:
            new_detections.append(detection)
            del detections[index]
    return new_detections

In [12]:
## resize
def resize_by_short(img, short_len=256):
    print(img.shape)
    (x, y) = img.shape
    if x > y:
        y_s = short_len
        x_s = int(x * y_s / y)
        img = cv2.resize(img, (x_s, y_s))
    else:
        x_s = short_len
        y_s = int(y * x_s / x)
        img = cv2.resize(img, (x_s, y_s))
    return img

def sliding_window(image, window_size, step_size):
    for y in range(0, image.shape[0], step_size[1]):  # 图片的函数为image.shape[0]
        for x in range(0, image.shape[1], step_size[0]): # 图片的列数为image.shape[1]
            yield (x, y, image[y:y+window_size[1], x:x+window_size[0]])

## 测试分类器
def test_classifier(config=config):
    for im_path in [os.path.join(config.DirPath["TEST_DIR_PATH"], i) for i in os.listdir(config.DirPath["TEST_DIR_PATH"])]:
        #img = Image.open(im_path).convert('L')
        img = cv2.imread(im_path, 0)
        img = np.array(resize_by_short(img))
        #return img
        # clf
        detections = []
        scale = 0
        
        print("Processing image %s, shape %s" % (im_path, str(img.shape)))
        ## 图像金字塔
        for img_scale in pyramid_gaussian(img, downscale=1.2):
            img_scale = (img_scale*255).astype(np.uint8)
#             plt.imshow(img_scale)
            
            cd = []
#             print("Img_scale shape %s" % (str(img_scale.shape)))
            #plt.imshow(img_scale)
            if img_scale.shape[0] < 64 or img_scale.shape[1] < 64:
                break
            for (x, y, img_window) in sliding_window(img_scale, [64,64], [12,12]):
                #print("Img_window shape : %s" % (str(img_window.shape)))
                if img_window.shape[0] != 64 or img_window.shape[1] != 64:
                    continue
                #plt.imshow(img_window)
                pred = Classifier(img_window)
                score = Classifier(img_window, True)
                if pred:  # 如果预测出来有目标
                    print("==> Location -> ({}, {})".format(x, y))
                    print("==> Scale -> {} Confidence Score {} ".format(scale, score))
                    detections.append((x, y, score, 
                                       int(64 * (1.2**scale)), 
                                       int(64 * (1.2**scale))))
                    cd.append(detections[-1])
                    
                clone = img_scale.copy()
                for x1, y1, _, _, _ in cd:
                    cv2.rectangle(clone, (x1, y1), (x1 + 64, y1 + 64), (0, 0, 0), thickness=2)
                cv2.rectangle(clone, (x, y), (x + 64, y + 64), (255,255,255), thickness=2)
                cv2.imshow("Sliding Window in Progress", clone)
                cv2.waitKey(30)
            scale += 1
        #Display the results before permorming NMS
        clone = img.copy()
        
        for (x_tl, y_tl, _, w, h) in detections:
            cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 0), thickness=2)

        detections_fin = nms(detections, 0.3) #perform Non Maxima Suppression
        
         # Display the results after performing NMS
        for (x_tl, y_tl, _, w, h) in detections_fin:
            # Draw the detections
            cv2.rectangle(clone, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 0), thickness=2)
        
        #cv2.imshow("Final Detections after applying NMS", clone)
        
        #保存结果
        print(os.path.join(config.DirPath['PRED_SAVE_PATH'], os.path.split(im_path)[1]))
        cv2.imwrite(os.path.join(config.DirPath['PRED_SAVE_PATH'], os.path.split(im_path)[1]), clone)
        
test_classifier(config)

(2320, 2320)
Processing image ./source/test_images/test_9.JPG, shape (256, 256)
==> Location -> (60, 72)
==> Scale -> 0 Confidence Score 2.1312403301142293 
==> Location -> (72, 72)
==> Scale -> 0 Confidence Score 1.545871986645812 
==> Location -> (96, 72)
==> Scale -> 0 Confidence Score 2.753934784141398 
==> Location -> (108, 72)
==> Scale -> 0 Confidence Score 2.411431295348171 
==> Location -> (120, 72)
==> Scale -> 0 Confidence Score 2.587697810239211 
==> Location -> (132, 72)
==> Scale -> 0 Confidence Score 4.351154874039144 
==> Location -> (144, 72)
==> Scale -> 0 Confidence Score 2.371092241371576 
==> Location -> (48, 84)
==> Scale -> 0 Confidence Score 2.4131554671892044 
==> Location -> (60, 84)
==> Scale -> 0 Confidence Score 2.4296722966280466 
==> Location -> (96, 84)
==> Scale -> 0 Confidence Score 2.132491435542539 
==> Location -> (108, 84)
==> Scale -> 0 Confidence Score 2.8553645073348033 
==> Location -> (120, 84)
==> Scale -> 0 Confidence Score 2.791623995816058

/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


==> Location -> (72, 48)
==> Scale -> 1 Confidence Score 1.7049845217959552 
==> Location -> (84, 48)
==> Scale -> 1 Confidence Score 2.6706940044480287 
==> Location -> (96, 48)
==> Scale -> 1 Confidence Score 2.071565481362091 
==> Location -> (108, 48)
==> Scale -> 1 Confidence Score 1.8906717819202237 
==> Location -> (60, 60)
==> Scale -> 1 Confidence Score 1.5598221356433493 
==> Location -> (84, 60)
==> Scale -> 1 Confidence Score 3.2898403701622154 
==> Location -> (108, 60)
==> Scale -> 1 Confidence Score 1.6012882223667526 
==> Location -> (48, 72)
==> Scale -> 1 Confidence Score 1.9341291630253232 
==> Location -> (60, 72)
==> Scale -> 1 Confidence Score 1.6141421705053796 
==> Location -> (72, 72)
==> Scale -> 1 Confidence Score 2.2881170718856136 
==> Location -> (84, 72)
==> Scale -> 1 Confidence Score 2.4719849198816117 
==> Location -> (96, 72)
==> Scale -> 1 Confidence Score 1.5588546878204903 
==> Location -> (72, 84)
==> Scale -> 1 Confidence Score 1.8626375295207305

/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


==> Location -> (60, 36)
==> Scale -> 2 Confidence Score 1.6863303980801498 
==> Location -> (72, 36)
==> Scale -> 2 Confidence Score 1.9100023291446229 
==> Location -> (48, 48)
==> Scale -> 2 Confidence Score 2.9871258000500855 
==> Location -> (60, 48)
==> Scale -> 2 Confidence Score 1.8759455670867788 
==> Location -> (72, 48)
==> Scale -> 2 Confidence Score 1.6005487334127844 
==> Location -> (48, 60)
==> Scale -> 2 Confidence Score 1.8544600561165276 
==> Location -> (72, 60)
==> Scale -> 2 Confidence Score 2.042494845763491 
==> Location -> (0, 84)
==> Scale -> 2 Confidence Score 2.4095350470199772 


...

/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


==> Location -> (72, 60)
==> Scale -> 4 Confidence Score 1.7634904740946291 


/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


./source/predictions/PROJECT_ORB_RGB/test_0.JPG


/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '
/home/ruisongzhou/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:24: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


In [ ]:
plt.imshow((temp*255).astype(np.uint8))

In [ ]:
img = cv2.imread("./source/test_images/pos_test_3.jpg", 0)
Classifier(img, True)

In [ ]:
plt.imshow(img)

In [ ]:
# for img_scale in pyramid_gaussian(img, downscale=1.2):
#     plt.imshow(img_scale)

### 去除文件名中的空格

In [ ]:
dirpath = "./source/images/neg_new/"
for file in os.listdir(dirpath):
    if ".mp4" in file:
        new_name = file.replace(".mp4", "")
        os.rename(os.path.join(dirpath, file), os.path.join(dirpath, new_name))

## ORB特征点检测

In [ ]:
path = "./source/images/pos/positiveSimples_VID_20181120_204145.mp4_10.jpg"
#path2 = "./source/images/neg_new/negativeSimples_VID_20181120_204145.mp4_1.jpg"
#path2 = "./source/test_images/test_6.jpg"

img1 = cv2.imread(path, 0)
# img2 = cv2.imread(path2, 0)

orb = cv2.ORB_create(nfeatures=200,
                     scaleFactor=1.2,
                     nlevels=8,
                     edgeThreshold=2,
                     WTA_K=2,
                     scoreType=cv2.ORB_HARRIS_SCORE,
                     patchSize=10)
kp1, des1 = orb.detectAndCompute(img1, None)
# kp2, des2 = orb.detectAndCompute(img2, None)

# bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# matches = bf.match(des1, des2)

img3 = cv2.drawKeypoints(img1, kp1, None, 255, 0)
# img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None)
plt.imshow(img3)

In [ ]:
des1.shape

In [ ]:
des2

In [ ]:
des

In [ ]:
len(kp)

In [ ]:
## 读取图片和保存图片路径
IMG_DIR_PATH = "./source/images/pos"
#ORB_IMG_SAVE_PATH = "./source/orb_feature/neg_new"

## 读取图片
ImgNames = os.listdir(IMG_DIR_PATH)
start_time = time.time()
for name in tqdm.tqdm(ImgNames):
    path = os.path.join(IMG_DIR_PATH, name)
    img = cv2.imread(path,0)
    ## 初始化检测器
    orb = cv2.ORB_create(nfeatures=50,
                         edgeThreshold=5)
    kp, des = orb.detectAndCompute(img, None)

    ## 画出关键点的位置
    img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
    cv2.imwrite(os.path.join(ORB_IMG_SAVE_PATH, name), img2)
end_time = time.time()
print("==> Process {} second." % end_time-start_time)

In [ ]:
#img = cv2.imread("./source/images/neg_new/negativeSimples__196.jpg",0)
img = cv2.imread("./source/images/pos/positiveSimples_VID_20181120_204145.mp4_101.jpg",0)
dst = cv2.cornerHarris(img,2,3,0.04)

dst = cv2.dilate(dst, None)
img[dst>0.01*dst.max()]=255
plt.imshow(img)